<a href="https://colab.research.google.com/github/JavIonBek/chatbot-uz/blob/main/chatbot_endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot - chatting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
import random

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pickle.load( open( "/content/drive/My Drive/Colab_Notebooks/Machine Learning Foundations Google/NLP/Chatbot/Chatbot-uz/chatbot-data-main.pkl", "rb" ) )
words = data['words']
classes = data['classes']

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
p = bow("Assalomu aleykum, yaxshimisiz", words)
print(p)
print(classes)

found in bag: assalomu
found in bag: aleyk
[0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['kim_san', 'qonun_buzarlikni_yuklash', 'rahmat', 'rezyume_yuborish', 'salom', 'xayr']


In [ ]:
global graph
graph = tf.compat.v1.get_default_graph()

In [ ]:
# From chatbot-model.ipynb
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils


def unpack(model, training_config, weights):
  restored_model = deserialize(model)
  if training_config is not None:
    restored_model.compile(
      **saving_utils.compile_args_from_training_config(
          training_config
      )
    )
  restored_model.set_weights(weights)
  return restored_model

In [ ]:
with open(f"/content/drive/MyDrive/Colab_Notebooks/Machine Learning Foundations Google/NLP/Chatbot/Chatbot-uz/chatbot-model-main.pkl", 'rb') as f:
  model = pickle.load(f)

In [ ]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [ ]:
classify_local('Salom, xayrli kun!')

found in bag: salom
found in bag: kun


[('salom', '0.9999871')]

In [ ]:
classify_local('Rahmat!')

found in bag: rahm


[('rahmat', '0.999949')]

In [ ]:
classify_local('Juda yordami tegdi, rahmat')

found in bag: yordam
found in bag: rahm


[('rahmat', '0.999974')]

In [ ]:
classify_local('Sog\' bo\'ling')

found in bag: sog
found in bag: '
found in bag: bo'ling


[('xayr', '0.9999933')]

In [ ]:
classify_local('Qalaysiz?')

found in bag: qalays


[('salom', '0.999871')]

In [ ]:
classify_local('Салом')

found in bag: салом


[('salom', '0.99972385')]

In [ ]:
classify_local('Qanday qilib qonun buzarlik haqida xabar berish mumkin?')

found in bag: qanday
found in bag: qilib
found in bag: qonun
found in bag: buzarlik
found in bag: haqid
found in bag: xab


[('qonun_buzarlikni_yuklash', '1.0')]

## Chatting

In [ ]:
# import our chat-bot intents file
import json
with open('/content/drive/MyDrive/Colab_Notebooks/Machine Learning Foundations Google/NLP/Chatbot/Chatbot-uz/intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
print(intents['intents'])

[{'tag': 'salom', 'patterns': ['Salom', 'Assalomu aleykum', 'Qalaysiz', 'Assalom', 'Hayrli kun', 'Салом', 'Aссалому алейкум', 'Aссалом'], 'responses': ['Sizga xursandman :) Nima bilan yordam berishim mumkin?', 'Assalomu aleykum, web saytimizga xush kelibsiz. Qanday yordam bera olaman?', 'Assalomu aleykum, qanday yordam bera olaman?'], 'context': ['']}, {'tag': 'xayr', 'patterns': ['Xayr', "Ko'rishguncha", "Sog' bo'ling", 'Хайр', 'Кўришгунча', 'Соғ бўлинг'], 'responses': ["Omad tilayman! Yana uchrashishdan xursand bo'laman.", "Sog' bo'ling, ko'rishguncha :)", "Kuningiz yaxshi o'tsin.", 'Xayr! Tashrifingizdan xursandmiz.'], 'context': ['']}, {'tag': 'rahmat', 'patterns': ['Rahmat', 'Raxmat', 'Kattakon rahmat', 'Yordamingiz uchun rahmat', 'Раҳмат', 'Рахмат', 'Каттакон раҳмат', 'Ёрдамингиз учун раҳмат'], 'responses': ['Men foydali bo`lishga harakat qilyapman.', 'Yordam berganimdan xursandman!', 'Arzimaydi.'], 'context': ['']}, {'tag': 'kim_san', 'patterns': ['Kim san?', 'Nima san?', 'Ismin

In [ ]:
def chat():
  print("Chat boshlandi!")
  while True:
    inp = input('Siz: ')
    if inp.lower() == 'stop':
      break
    
    inp = pd.DataFrame([bow(inp, words, show_details=False)], dtype=float, index=['input'])
    results = model.predict([inp])[0]
    results_index = np.argmax(results)
    tag = classes[results_index]

    if results[results_index] > 0.7:
      for tg in intents['intents']:
        if tg['tag'] == tag:
          responses = tg['responses']
          print('Bot:', random.choice(responses))
    else:
      print("Bot: Afsuski, Sizga yordam bera olmadim. Mutaxassis bilan bog’lanishni istaysizmi?\n1. Ha\n2. Yo'q")
      # Afsuski, Sizga yordam bera olmadim. Mutaxassis bilan bog’lanishni istaysizmi?
      # Ha
      # Yo’q
      # Kechirasiz, ehtimol men Sizni noto`g`ri tushundim. Iltimos, savolni boshqacha bering va men javob berishga harakat qilaman.


chat()

Chat boshlandi!
Siz: salom
Bot: Sizga xursandman :) Nima bilan yordam berishim mumkin?
Siz: Qanday qilib qunun buzarlik haqida xabar berish mumkin?
Bot: Saytning Qonun buzarliklar sahifasiga o'tib, tegishli ma'lumotlarni kiritib, ma'limot yuklashingiz mumkin.
Siz: rahmat
Bot: Arzimaydi.
Siz: dfgdfgdfgdf
Bot: Afsuski, Sizga yordam bera olmadim. Mutaxassis bilan bog’lanishni istaysizmi?
1. Ha
2. Yo'q
Siz: salom
Bot: Assalomu aleykum, qanday yordam bera olaman?
Siz: xayr
Bot: Xayr! Tashrifingizdan xursandmiz.
Siz: stop
